In [46]:
!pip install --upgrade --quiet  langchain-core langchain-community langchain-openai langchain-groq langchain neo4j python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import pandas as pd
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain.chains import GraphCypherQAChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
import os
from dotenv import load_dotenv
load_dotenv()

groq_api = os.getenv("GROQ_API_KEY")

# Neo4j 
neo4j_url = os.getenv("NEO4J_CONNECTION_URL")
neo4j_user = os.getenv("NEO4J_USER")
neo4j_password = os.getenv("NEO4J_PASSWORD")

# https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html
graph = Neo4jGraph(neo4j_url,neo4j_user,neo4j_password)

# Prepare Data

In [2]:
df = pd.read_csv('/content/data_obat_fix_ordered (2).csv', sep = ',')
df = df[['id','nomor_registrasi','types','prepro_indications','cleaned_names']]
bpom = pd.read_csv('/content/bpom_full.csv')
merge= pd.merge(df, bpom, on='nomor_registrasi', how='left')[['id','nomor_registrasi','types','prepro_indications','cleaned_names','nama_produk','komposisi','klaim']]
merge['komposisi'] = merge['komposisi'].astype(str).apply(lambda x: x.lower().replace('- ', '').replace('.', '|'))
merge['nama'] = merge['nama_produk'].astype(str).apply(lambda x: x.lower())
merge['prepro_indications'] = merge['prepro_indications'].fillna(merge['klaim'])
merge = merge[['id','nomor_registrasi','nama','types','komposisi','prepro_indications']]
merge.drop_duplicates(subset=['nomor_registrasi', 'nama'], inplace = True)
merge = merge.sort_values(by = 'id').reset_index(drop=True)
merge

,id,nomor_registrasi,nama,types,komposisi,prepro_indications
0,0,GKL1433531717A1,sildenafil citrate,tablet,sildenafil citrate,terapi disfungsi ereksi pria dewasa
1,5,DKI1690401417A1,viagra,tablet,sildenafil citrate,terapi disfungsi ereksi pria dewasa
2,7,DKL1333528917A1,ericfil,tablet,sildenafil citrate,ericfil diindikasikan mengobati disfungsi ereksi
3,9,DKI1973401817C1,cialis,tablet,tadalafil,pengobatan ketidakmampuan mencapai mempertahan...
4,12,DKL2233554519A1,ericfil 50,odf,sildenafil citrate,terapi disfungsi ereksi pria dewasa
...,...,...,...,...,...,...
4749,10053,TR132474581,sinargi astramuno,pil,astragalus membranaceus radix,membantu memelihara kesehatan
4750,10054,TR213603191,probiomag,madu,ekstrak cinamommum burmani cortex,"meringankan gangguan pencernaan , perut mual k..."
4751,10055,TR183613971,madu gurah fit,madu,abrus precatorius folium extract,membantu meredakan batuk
4752,10056,TR183622691,madu nurutenz,madu,apium graveolens herba ekstrak,meringankan gejala hipertensi


In [16]:
model = ChatGroq(model="llama3-70b-8192",groq_api_key = groq_api)

def process_indikasi(text):
  template = """
      Extract the types of diseases that can be cured from a given text.
      If more than one, separate them with '|', do not use any other symbols.
      Do not add any explanations, reasoning or preambles, if none exist then return 'None'.
      Your output should be in Bahasa Indonesia

      Example:

      text = "membantu meringankan tekanan darah ringan"
      helpful answer = "tekanan darah ringan"

      text = "pengobatan ketidakmampuan mencapai mempertahankan ereksi berhubungan seksual disfungsi ereksi pria dewasa , mengobati penderita pembesaran prostat jinak bph hipertensi pulmonal tekanan pembuluh darah arteri paru - paru"
      helpful answer = "disfungsi ereksi|pembesaran prostat jinak|hipertensi|tekanan pembuluh darah arteri paru-paru"

      text: {text}
      helpful answer:
      """

  prompt = ChatPromptTemplate.from_template(template = template)
  output_parser = StrOutputParser()

  chain = prompt | model | output_parser

  result = chain.invoke({"text": text})
  print(result)
  return result

In [14]:
result = process_indikasi("mengurangi kejadian aterosklerosis infark miokard , stroke kematian vaskular pasien aterosklerosis ditandai stroke , infark miokard penyakit arteri")

disfungsi ereksi|pembesaran prostat|hipertensi|infark miokard|stroke|penyakit arteri


In [ ]:
merge

,id,nomor_registrasi,nama,types,komposisi,prepro_indications
0,0,GKL1433531717A1,sildenafil citrate,tablet,sildenafil citrate,terapi disfungsi ereksi pria dewasa
1,5,DKI1690401417A1,viagra,tablet,sildenafil citrate,terapi disfungsi ereksi pria dewasa
2,7,DKL1333528917A1,ericfil,tablet,sildenafil citrate,ericfil diindikasikan mengobati disfungsi ereksi
3,9,DKI1973401817C1,cialis,tablet,tadalafil,pengobatan ketidakmampuan mencapai mempertahan...
4,12,DKL2233554519A1,ericfil 50,odf,sildenafil citrate,terapi disfungsi ereksi pria dewasa
...,...,...,...,...,...,...
4749,10053,TR132474581,sinargi astramuno,pil,astragalus membranaceus radix,membantu memelihara kesehatan
4750,10054,TR213603191,probiomag,madu,ekstrak cinamommum burmani cortex,"meringankan gangguan pencernaan , perut mual k..."
4751,10055,TR183613971,madu gurah fit,madu,abrus precatorius folium extract,membantu meredakan batuk
4752,10056,TR183622691,madu nurutenz,madu,apium graveolens herba ekstrak,meringankan gejala hipertensi


In [25]:
sample = merge.sample(frac=10, replace = True).sample(n=100)
sample

,id,nomor_registrasi,nama,types,komposisi,prepro_indications
1621,3514,DKL1007810604A1,hufadon,kaplet,domperidone,terapi mual muntah disebabkan levodopa bromokr...
4175,8759,DKL0804520010A1,norizec,tablet,glimepiride,diabetes melitus tipe 2 non - insulin - depend...
4356,9097,DKI0367501817C1,diovan,tablet,valsartan,"hipertensi , gagal jantung , pasca infark miokard"
2165,4681,DKL1203813646A1,lentikular,tetes,pirenoxine,membantu mengatasi kekeringan mata
287,926,DKL0804130337A1,comtusi (rasa strawberry),sirup,oxomemazine| guaifenesin,"untuk meringankan gejala batuk alergi , batuk ..."
...,...,...,...,...,...,...
3324,6728,DKL0304421302B1,oscal,kapsul,calcitriol,osteoporosis pasca menopause osteodistrofi re...
3090,6354,DKL0609215704A1,rexavin,kaplet,griseofulvin,NaN
1683,3612,DKL1733537682A1,monell,tablet,domperidone,"mual muntah akut , dispepsia gangguan pencerna..."
2463,5356,DKL0434601629A1,afucid,krim,fusidic acid,"impetigo kontagiosum , folikulitis superfisial..."


In [26]:
sample['indikasi'] = sample['prepro_indications'].map(process_indikasi)

mual muntah|kanker|dispepsia fungsional
diabetes melitus tipe 2|non-insulin-dependent type ii diabetes melitus|niddm
hipertensi|gagal jantung|infark miokard
kekeringan mata
batuk alergi|batuk berdahak
asma bronkial|bronkritis kronik|emfisema
alergi saluran pernapasan|alergi kulit|mata
None
None
batuk|demam|sakit kepala|hiding bersin-bersin
alergi rinitis nasal|rinitis intermiten persisten|pruritus|urtikaria khronik idiopathik
Here is the answer:

infeksi sal cerna|infeksi sal nafas|infeksi bakteri
hipertensi|penyakit arteri koroner
kandidiasis mukokutan khronis|infeksi jamur sistemik
hipertensi|gagal jantung kronik
batuk pilek
batuk
disfungsi ereksi|pembesaran prostat jinak|hipertensi|tekanan pembuluh darah arteri paru-paru
None
sakir kepala|sakit gigi|dismenore primer|nyeri trauma|nyeri otot|nyeri operasi
infeksi saluran kemih|otitis medium|radang rongga gendang telinga|faringitis|tonsilitis|bronkhitis akut|bronkhitis kronis
sakit kepala
dehidrasi ringan|diare|muntah
hipsterplasia pro

In [44]:
sample.to_csv('sample_skripsi.csv', index = False)

In [36]:
import numpy as np

def filter_parts(value):
    if value == 'None':
        return np.nan
    split_values = value.lower().split(':')
    filtered_parts = [part for part in split_values if '|' in part]
    return '|'.join(filtered_parts) if filtered_parts else value.lower()

# Create a new column with filtered parts
sample['indikasi'] = sample['indikasi'].map(filter_parts)
df['indikasi'] = df['indikasi'].str.replace('"', '').str.replace('\n', '')

In [37]:
sample

,id,nomor_registrasi,nama,types,komposisi,prepro_indications,indikasi,filtered_indikasi
1621,3514,DKL1007810604A1,hufadon,kaplet,domperidone,terapi mual muntah disebabkan levodopa bromokr...,mual muntah|kanker|dispepsia fungsional,mual muntah|kanker|dispepsia fungsional
4175,8759,DKL0804520010A1,norizec,tablet,glimepiride,diabetes melitus tipe 2 non - insulin - depend...,diabetes melitus tipe 2|non-insulin-dependent ...,diabetes melitus tipe 2|non-insulin-dependent ...
4356,9097,DKI0367501817C1,diovan,tablet,valsartan,"hipertensi , gagal jantung , pasca infark miokard",hipertensi|gagal jantung|infark miokard,hipertensi|gagal jantung|infark miokard
2165,4681,DKL1203813646A1,lentikular,tetes,pirenoxine,membantu mengatasi kekeringan mata,kekeringan mata,kekeringan mata
287,926,DKL0804130337A1,comtusi (rasa strawberry),sirup,oxomemazine| guaifenesin,"untuk meringankan gejala batuk alergi , batuk ...",batuk alergi|batuk berdahak,batuk alergi|batuk berdahak
...,...,...,...,...,...,...,...,...
3324,6728,DKL0304421302B1,oscal,kapsul,calcitriol,osteoporosis pasca menopause osteodistrofi re...,osteoporosis|osteodistrofi renal|gangguan ginj...,osteoporosis|osteodistrofi renal|gangguan ginj...
3090,6354,DKL0609215704A1,rexavin,kaplet,griseofulvin,NaN,NaN,NaN
1683,3612,DKL1733537682A1,monell,tablet,domperidone,"mual muntah akut , dispepsia gangguan pencerna...",mual muntah akut|mual muntah anak akibat kemot...,mual muntah akut|mual muntah anak akibat kemot...
2463,5356,DKL0434601629A1,afucid,krim,fusidic acid,"impetigo kontagiosum , folikulitis superfisial...",impetigo kontagiosum|folikulitis superfisial|f...,impetigo kontagiosum|folikulitis superfisial|f...


In [38]:
sample.rename(columns={'types': 'tipe'}, inplace=True)
sample.drop(['prepro_indications'], axis=1, inplace=True)
sample

In [43]:
sample

,id,nomor_registrasi,nama,tipe,komposisi,indikasi
1621,3514,DKL1007810604A1,hufadon,kaplet,domperidone,mual muntah|kanker|dispepsia fungsional
4175,8759,DKL0804520010A1,norizec,tablet,glimepiride,diabetes melitus tipe 2|non-insulin-dependent ...
4356,9097,DKI0367501817C1,diovan,tablet,valsartan,hipertensi|gagal jantung|infark miokard
2165,4681,DKL1203813646A1,lentikular,tetes,pirenoxine,kekeringan mata
287,926,DKL0804130337A1,comtusi (rasa strawberry),sirup,oxomemazine| guaifenesin,batuk alergi|batuk berdahak
...,...,...,...,...,...,...
3324,6728,DKL0304421302B1,oscal,kapsul,calcitriol,osteoporosis|osteodistrofi renal|gangguan ginj...
3090,6354,DKL0609215704A1,rexavin,kaplet,griseofulvin,NaN
1683,3612,DKL1733537682A1,monell,tablet,domperidone,mual muntah akut|mual muntah anak akibat kemot...
2463,5356,DKL0434601629A1,afucid,krim,fusidic acid,impetigo kontagiosum|folikulitis superfisial|f...


# Insert to Neo4J

In [4]:
graph.refresh_schema()
print(graph.schema)

Node properties:

Relationship properties:

The relationships:



In [13]:
df = pd.read_csv('sample_skripsi.csv')



In [8]:
insert_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/projectwilsen/skripsi/main/sample_skripsi.csv'
AS row
MERGE (obat:Obat {nama: row.nama, id: row.id, no_regis: row.nomor_registrasi})
MERGE (tipe:Tipe {nama: row.tipe})

FOREACH (kom in split(row.komposisi, '|') |
    MERGE (komposisi:Komposisi {nama:trim(kom)})
    MERGE (obat)-[:MENGANDUNG]->(komposisi))

FOREACH (ind in split(row.indikasi, '|') |
    MERGE (indikasi:Indikasi {nama:trim(ind)})
    MERGE (obat)-[:MENGOBATI]->(indikasi))

MERGE (obat)-[:TIPE]->(tipe)
"""

graph.query(insert_query)

ClientError: {code: Neo.ClientError.Statement.SemanticError} {message: Cannot merge the following node because of null property value for 'nama': (:Tipe {nama: null}) (Failure when processing file 'sample_skripsi.csv' on line 9.)}